<a href="https://colab.research.google.com/github/MaximTislenko/DS_in_MED/blob/main/Less_05/Tutorial_3_pyhealth_models_RUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Подготовка**
- установить альфа-версию pyhealth

In [ ]:
!pip install pyhealth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **Инструкция по [pyhealth.models](https://pyhealth.readthedocs.io/en/latest/api/models.html)**
- **[README]**: В этом пакете мы предоставляем общие модели глубокого обучения. (например: RNN, CNN, Transformer) и специальные модели глубокого обучения в сфере здравоохранения (например: RETAIN, SafeDrug, GAMENet).. Все модели, за исключением некоторых специальных (например, GAMENet, SafeDrug, MICRON, предназначены только для задач по рекомендации лекарств), могут применяться ко всем задачам прогнозирования состояния здоровья. Обратите внимание, что мы предоставили два вызываемых метода для каждой модели глубокого обучения:
  - Модель, такая как RNN, CNN, Transformer, RETAIN **инициализированная нашим объектом набора данных**.
  - ModelLayer (альтернативно), например RNNLayer, CNNLayer, TransformerLayer, RETAINLayer. Альтернативно, **инициализируется вспомогательной информацией (указанной для каждого уровня)**.

- **[Arguments for Model]**:
  Аргументы для каждой модели DL соответствуют приведенным ниже аргументам.
    - `dataset`: это объект [pyhealth.dataset](https://pyhealth.readthedocs.io/en/latest/api/datasets.html). Вся обработка, специфичная для модели, основана на этом и обрабатывается в классе Model.
    - `feature_keys`: список имен таблиц на основе строк, указывающий, что эти таблицы будут использоваться в конвейере.
    - `label_key`: в настоящее время мы поддерживаем только метку (`label`), определенную в функции задачи.
       - `mode`: `multiclass`, `multilabel`, or `binary`.
    
- **[Arguments for the ModelLayer]**:
В качестве альтернативы, если пользователи не хотят использовать объект [pyhealth.dataset](https://pyhealth.readthedocs.io/en/latest/api/datasets.html) для инициализации модели, они могут выбрать вызов ModelLayer, подготовив входные данные в соответствии с требованиями. Входные данные каждого ModelLayer могут быть разными (см. [pyhealth.models](https://pyhealth.readthedocs.io/en/latest/api/models.html), например мы перечисляем аргументы RNNLayer ниже:
  - `input_size`: входной размер rnn
  - `hidden_size`: скрытый размер rnn
  - `rnn_type`: тип rnn, например: GRU, LSTM
  - `num_layers`: количество слоев rnn
  - `dropout`: уровень отсечения
  - `bidirectional`: использовать ли двунаправленный rnn


### **Шаг 1 и 2: Подготовьте наборы данных и функцию задачи**
- Мы используем **набор данных OMOP** для задачи **прогнозирования повторной госпитализации**. Более подробную информацию можно найти в [Уроке 1](https://colab.research.google.com/drive/18kbzEQAj1FMs_J9rTGX8eCoxnWdx4Ltn?usp=sharing) и [Уроке 2](https://colab.research.google.com/drive/1r7MYQR_5yCJGpK_9I9-A10HmpupZuIN-?usp=sharing).

In [ ]:
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.tasks import readmission_prediction_mimic3_fn

dataset = MIMIC3Dataset(
        root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
        code_mapping={"NDC": "ATC"},
        # режим разработки — True, включает небольшую загрузку данных.
        dev=True,
)

# уточните какая задача
dataset = dataset.set_task(readmission_prediction_mimic3_fn)

Generating samples for readmission_prediction_mimic3_fn: 100%|██████████| 1000/1000 [00:00<00:00, 157722.11it/s]


In [ ]:
# проверьте формат первого образца
dataset.samples[2]

{'visit_id': '100183',
 'patient_id': '175',
 'conditions': [['5990',
   '4280',
   '2851',
   '4240',
   '2749',
   '9982',
   'E8499',
   '42831',
   '34600']],
 'procedures': [['0040', '3931', '7769']],
 'drugs': [['N06DA02',
   'V06DC01',
   'B01AB01',
   'A06AA02',
   'R03AC02',
   'H03AA01',
   'J01FA09']],
 'label': 0}

### **Шаг 3 (Пример): Использование RETAIN или RETAIN Layer**
- Опция 1: мы решили инициализировать модель **pyhealth.models.RETAIN**.
- Опция 2: мы решили настроить новую модель с помощью нашего **pyhealth.models.RETAINLayer**.


In [ ]:
# Опция 1

from pyhealth.models import RETAIN

device = "cpu"

model = RETAIN(
    # аргумент 1: вызвать набор данных
    dataset=dataset,
    # аргумент 2: используйте подмножество ключей в формате образца данных для функций
    # посмотрите, что доступно для «feature_keys» и «label_keys» в dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    # аргумент 3: используйте `label` для указания прогноза label_key
    label_key="label",
    # аргумент 4: установите размер внедрения
    embedding_dim=128,
    # аргумент 5: какой тип задач: мультиклассовый, мультиметочный или двоичный?
    mode="binary",
)
model.to(device)

RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(303, 128, padding_idx=0)
    (procedures): Embedding(101, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [ ]:
# Опция 2

from pyhealth.models import RETAINLayer
import torch.nn as nn

class NewModel(nn.Module):
    def __init__(
        self,
        input_size: int = 64,
        hidden_size: int = 128,
        num_layers: int = 1,
        dropout: float = 0.5,
    ):
        super(NewModel, self).__init__()

        # TODO: implement other module 1
        self.module1 = None

        # initilize the RNNLayer
        self.rnn = RETAINLayer(input_size, dropout)

        # TODO: implement other module 2
        self.module2 = None


    def forward(self, x):
        x = self.module1(x)
        # call the RNNLayer
        x = self.rnn(x)
        x = self.module2(x)
        return x

model = NewModel()

### **Шаг 4 (Пример): Использование модели Трансформера**
- Опция 1: мы выбираем инициализацию модели **pyhealth.models.Transformer**.
- Опция 2: мы решили настроить новую модель с помощью нашего **pyhealth.models.TransformerLayer**.


In [ ]:
# Опция 1

from pyhealth.models import Transformer

device = "cpu"

model = Transformer(
    # аргумент 1: вызвать набор данных
    dataset=dataset,
    # аргумент 2: используйте подмножество ключей в формате образца данных для функций
    # посмотрите, что доступно для «feature_keys» и «label_keys» в dataset.samples[0]
    feature_keys=["conditions", "procedures"],
    # аргумент 3: используйте `label` для указания прогноза label_key
    label_key="label",
    # аргумент 4: установите размер внедрения
    embedding_dim=128,
    # аргумент 5: какой тип задач: мультиклассовый, мультиметочный или двоичный?
    mode="binary",
)
model.to(device)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(303, 128, padding_idx=0)
    (procedures): Embedding(101, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=128, out_features=128, bias=False)
              (1): Linear(in_features=128, out_features=128, bias=False)
              (2): Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, 

In [ ]:
# вариант 2: создайте свою новую модель

from pyhealth.models import TransformerLayer
import torch.nn as nn

class NewModel(nn.Module):
    def __init__(
        self,
        input_size: int = 64,
        hidden_size: int = 128,
        num_layers: int = 1,
        dropout: float = 0.5,
    ):
        super(NewModel, self).__init__()

        # вы можете реализовать другие модули здесь
        self.module1 = None

        # инициализировать уровень RNN
        self.transformer = TransformerLayer(input_size, dropout)

        # вы можете реализовать другие модули здесь
        self.module2 = None


    def forward(self, x):
        x = self.module1(x)
        # позвоните в RNNLayer
        x = self.transformer(x)
        x = self.module2(x)
        return x

model = NewModel()

If you find it useful, please give us a star ⭐ (fork, and watch) at https://github.com/sunlabuiuc/PyHealth.

Thanks very much for your support!